<a href="https://colab.research.google.com/github/milton-miranda/Classificacao-desbalanceada/blob/main/Base1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install.package(c("tidyverse", "MASS", "caTools", "caret", "ggplot2", "UBL"))
install.package(c("smotefamily", "boot", "ROCR", "pROC", "gridExtra", "DMwR"))
install.package(c("e1071", "mltools", "tictoc"))

ERROR: Error in install.package(c("tidyverse", "MASS", "caTools", "caret", "ggplot2", : could not find function "install.package"


In [ ]:
library(tidyverse)
library(MASS)
library(caTools)
library(caret)
library(ggplot2)
library(UBL)
library(smotefamily)
library(boot)
library(ROCR)
library(pROC)
library(gridExtra)
library(DMwR)
library(e1071)
library(mltools)
library(tictoc)

base_dados <- TaiwanCredit
data_balanced_sub$Resposta <- as.factor(data_balanced_sub$Resposta)
data_balanced_hibrido$Resposta <- as.factor(data_balanced_hibrido$Resposta)
x<- base_dados %>% select(X6,X7,X8,X9,X10,X11)
vetor_transformado = list()
for (i in 1:ncol(x)) {
  avaliacao = ifelse(x[i] > 0,1,0)
  vetor_transformado[[i]] = avaliacao
}
var_transformada <- data.frame(vetor_transformado)

## Proporcao de pagamento.
y<- base_dados %>% select(X13,X14,X15,X16,X18,X19,X20,X21)
transforma_zero <- function(x){
  ifelse(is.nan(x)|is.infinite(x),0,x)
}
transforma_inconsistencia <- function(x){
  ifelse(x>1.5,1.5,x)
}
prop <- data.frame(rowSums(y[5:8])/rowSums(y[1:4]))
prop_pgto <- apply(prop,1,transforma_zero)
#prop_pgto_corrigido <- apply(data.frame(prop_pgto),1,transforma_inconsistencia)


## Construcao e separacao de variaveis
base_analise <- base_dados %>% transmute(
  Atraso_medio = (X6+X7+X8+X9+X10+X11)/6,
  faturaM_pagamento = rowSums(base_dados[, c('X18','X19','X20','X21','X22','X23')] > (X12+X13+X14+X15+X16+X17)/6),
  Prop_pagamento = prop_pgto,
  Quantidade_atrasos = rowSums(var_transformada),
  Resposta = as.factor(ifelse(Y == 0,-1,1))
)

rm(avaliacao,var_transformada,vetor_transformado,x,y,prop,prop_pgto,
   transforma_inconsistencia,transforma_zero,i)

################################################################################
#tratamento outliers
# Calcule o terceiro quartil (Q3)
Q3_1 <- quantile(base_analise$Atraso_medio, 0.75)
Q3_2 <- quantile(base_analise$faturaM_pagamento, 0.75)
Q3_3 <- quantile(base_analise$Prop_pagamento, 0.75)
Q3_4 <- quantile(base_analise$Quantidade_atrasos, 0.75)

Q1_1 <- quantile(base_analise$Atraso_medio, 0.25)
Q1_2 <- quantile(base_analise$faturaM_pagamento, 0.25)
Q1_3 <- quantile(base_analise$Prop_pagamento, 0.25)
Q1_4 <- quantile(base_analise$Quantidade_atrasos, 0.25)

# Calcule o limite superior para identificar outliers
IQR_1 <- IQR(base_analise$Atraso_medio)
IQR_2 <- IQR(base_analise$faturaM_pagamento)
IQR_3 <- IQR(base_analise$Prop_pagamento)
IQR_4 <- IQR(base_analise$Quantidade_atrasos)

limite_superior_1 <- Q3_1 + 1.5 * IQR_1
limite_superior_2 <- Q3_2 + 1.5 * IQR_2
limite_superior_3 <- Q3_3 + 1.5 * IQR_3
limite_superior_4 <- Q3_4 + 1.5 * IQR_4

limite_inferior_1 <- Q1_1 - 1.5 * IQR_1
limite_inferior_2 <- Q1_2 - 1.5 * IQR_2
limite_inferior_3 <- Q1_3 - 1.5 * IQR_3
limite_inferior_4 <- Q1_4 - 1.5 * IQR_4

# Substitua os outliers pelo valor de Q3
base_analise$Atraso_medio[base_analise$Atraso_medio > limite_superior_1] <- limite_superior_1
base_analise$faturaM_pagamento[base_analise$faturaM_pagamento > limite_superior_2] <- limite_superior_2
base_analise$Prop_pagamento[base_analise$Prop_pagamento > limite_superior_3] <- limite_superior_3
base_analise$Quantidade_atrasos[base_analise$Quantidade_atrasos > limite_superior_4] <- limite_superior_4

base_analise$Atraso_medio[base_analise$Atraso_medio < limite_inferior_1] <- limite_inferior_1
base_analise$faturaM_pagamento[base_analise$faturaM_pagamento < limite_inferior_2] <- limite_inferior_2
base_analise$Prop_pagamento[base_analise$Prop_pagamento < limite_inferior_3] <- limite_inferior_3
base_analise$Quantidade_atrasos[base_analise$Quantidade_atrasos < limite_inferior_4] <- limite_inferior_4
################################################################################
#descritiva
graficos_descritva <- function(base_dados){

  graficos <- list()
  Indice = NULL
  Variavel = NULL
  Tipo = NULL

  for (i in 1:(length(base_dados)-1)) {
    if (class(pull(base_dados,i))== "factor" |
        class(pull(base_dados,i))== "character"){
      dados_grafico <- data.frame(Resposta = base_dados$Resposta,
                                  Variavel = pull(base_dados,i))
      graficos[[i]] <- ggplot(data = dados_grafico,
                              aes(x = Variavel,
                                  y = prop.table(stat(count)),
                                  fill = Resposta,
                                  label = scales::percent(prop.table(stat(count)),
                                                          accuracy = 1))) +
        geom_bar(position=position_dodge()) +
        geom_text(stat = 'count',
                  position = position_dodge(.9),
                  vjust = -0.5,
                  size = 3.5) +
        scale_y_continuous(labels = scales::percent)+
        scale_fill_manual(values = c("darkolivegreen4", "tomato2"),
                          labels=c("Adimplente", "Inadimplente"))+
        labs(x = "", y = 'Frequência relativa (%)', fill = "Classificação")+
        ggtitle(names(base_dados[i]))+
        theme_bw()
      Variavel[i] = names(base_dados)[i]
      Tipo[i] = "Categorica"
    }else{
      dados_grafico <- data.frame(Resposta = base_dados$Resposta,
                                  Variavel = pull(base_dados,i))
      graficos[[i]] <- ggplot(data = dados_grafico, aes(x= Resposta, y = Variavel, fill = Resposta)) +
        geom_boxplot(color = "dimgray") +
        scale_x_discrete(labels=c("Adimplente", "Inadimplente")) +
        scale_fill_manual(values=c("darkolivegreen4", "tomato2"))+
        labs(x = "",y= names(base_dados[i]))+
        ggtitle(names(base_dados[i]))+
        theme_bw()+
        theme(legend.position= "none")
      Variavel[i] = names(base_dados)[i]
      Tipo[i] = "Numérica"
    }
    Indice[i]= i
  }
  guia_graficos <-  data.frame(Indice,Variavel,Tipo)
  lista_graficos <- list(graficos=graficos,guia_graficos = guia_graficos)
  return(lista_graficos)
}

novos_nomes <- c("Atraso m?dio", "Pagamento maior que fatura", "Propor??o de pagamento", "Quantidade de atrasos","Resposta")
colnames(base_analise) <- novos_nomes

plot_graficos <- graficos_descritva(base_dados = base_analise)
grid.arrange(plot_graficos$graficos[[1]], plot_graficos$graficos[[2]], plot_graficos$graficos[[3]], plot_graficos$graficos[[4]], nrow = 2, ncol = 2)
################################################################################
############ TREINO E TESTE ##################
#denominando as classes
inadimp <- base_analise %>% filter(Resposta==1)
adimp <- base_analise %>% filter(Resposta==-1)
set.seed(123)
#dividindo a base
ind = sample.split(Y = base_analise$Resposta, SplitRatio = 0.7)
#subsetting into Train data
treino <- base_analise[ind,]
treino_adimp <- treino %>% filter(Resposta==-1)
treino_inadimp <- treino %>% filter(Resposta==1)
#subsetting into Test data
teste = base_analise[!ind,]
#write.csv(treino, file = "treino_base1.csv")
#write.csv(teste, file = "teste_base1.csv")
############ REGRESSAO LOGISTICA DESBALANCEADA ##################
#regressao logistica no conjunto de treinamento
tic()
model <- glm(Resposta ~Atraso_medio+faturaM_pagamento+Prop_pagamento+Quantidade_atrasos,family=binomial(link='logit'),data=treino)
toc()
summary(model)
#predicao no conjunto de teste
p_samp <- predict(model, newdata=subset(teste), type="response")
p_samp

predicoes_totais <- predict(model, type = "response")
classe1 <- (treino$Resposta == 1)
predicoes_classe1 <- predicoes_totais[classe1]
summary(predicoes_classe1)
boxplot(model$fitted.values ~ treino$Resposta)

#KS
pred_modelo_geral = prediction(p_samp, teste$Resposta)

perf_modelo_geral = performance(pred_modelo_geral,"tpr","fpr")

TPRfromROCR=unlist(perf_modelo_geral@y.values)
FPRfromROCR=unlist(perf_modelo_geral@x.values)

diff_TPRFPR=TPRfromROCR-FPRfromROCR
KS=max(diff_TPRFPR)
cutoffAtKS=unlist(perf_modelo_geral@alpha.values)[which.max(diff_TPRFPR)]
print(c(KS, cutoffAtKS))

#matriz de cunfusao
previsoes_binarias <- ifelse(p_samp > 0.26, '1', '-1')
C_Matrix_Geral <- confusionMatrix(data = as.factor(previsoes_binarias),
                                  reference = teste$Resposta,
                                  positive = '1',
                                  mode = "everything");C_Matrix_Geral
auc(teste$Resposta,p_samp)
g_media <- sqrt(C_Matrix_Geral$byClass["Sensitivity"]*C_Matrix_Geral$byClass["Specificity"])
#MCC
teste_2 <- data.frame(p_samp, teste$Resposta)
class_predict <- ifelse(teste_2$p_samp > 0.26,1,-1) %>% as.factor()
MCC1 <- mcc(preds = class_predict, actuals =  teste_2$teste.Resposta);MCC1

########## BUSCA MELHOR PTCO CORTE
calcular_MCC_para_pontos_corte <- function(data, A, ponto_corte_seq) {
  melhores_MCC <- -Inf
  melhor_ponto_corte <- NA

  for (ponto_corte in ponto_corte_seq) {
    class_predict <- ifelse(data$p_samp > ponto_corte, 1, 0) %>% as.factor()
    MCC <- mcc(preds = class_predict, actuals = A$teste.Resposta)

    if (MCC > melhores_MCC) {
      melhores_MCC <- MCC
      melhor_ponto_corte <- ponto_corte
    }
  }

  return(list(melhor_ponto_corte = melhor_ponto_corte, melhor_MCC = melhores_MCC))
}

teste_2 <- data.frame(p_samp, teste$Resposta)
ponto_corte_seq <- seq(0, 0.99, 0.01)
resultado <- calcular_MCC_para_pontos_corte(teste_2, teste_2, ponto_corte_seq)

print(paste("Melhor Ponto de Corte:", resultado$melhor_ponto_corte))
print(paste("Melhor Valor de MCC:", resultado$melhor_MCC))

############ REGRESSAO LOGISTICA SMOTE ##################
table(treino$Resposta)
tic()
data_balanced_smote <- SmoteClassif(Resposta ~ ., treino, C.perc = list('-1' = 1, '1' = 3.52))
modelo_smote <- glm(Resposta ~Atraso_medio+faturaM_pagamento+Prop_pagamento+Quantidade_atrasos,family=binomial(link='logit'),data=data_balanced_smote)
toc()
table(data_balanced_smote$Resposta)
summary(modelo_smote)
#predicao no conjunto de teste
p_samp_smote <- predict(modelo_smote, newdata=subset(teste), type="response")
p_samp_smote

predicoes_totais <- predict(modelo_smote, type = "response")
classe1 <- (data_balanced_smote$Resposta == 1)
predicoes_classe1 <- predicoes_totais[classe1]
summary(predicoes_classe1)
boxplot(modelo_smote$fitted.values ~ data_balanced_smote$Resposta)

#KS
pred_modelo_geral_smote = prediction(p_samp_smote, teste$Resposta)

perf_modelo_geral_smote = performance(pred_modelo_geral_smote,"tpr","fpr")

TPRfromROCR_smote=unlist(perf_modelo_geral_smote@y.values)
FPRfromROCR_smote=unlist(perf_modelo_geral_smote@x.values)

diff_TPRFPR_smote=TPRfromROCR_smote-FPRfromROCR_smote
KS_smote=max(diff_TPRFPR_smote)
cutoffAtKS_smote=unlist(perf_modelo_geral_smote@alpha.values)[which.max(diff_TPRFPR_smote)]
print(c(KS_smote, cutoffAtKS_smote))

#matriz de cunfusao
previsoes_binarias_smote <- ifelse(p_samp_smote > 0.55, '1', '-1')
C_Matrix_Geral_smote <- confusionMatrix(data = as.factor(previsoes_binarias_smote),
                                        reference = teste$Resposta,
                                        positive = '1',
                                        mode = "everything");C_Matrix_Geral_smote
auc(teste$Resposta,p_samp_smote)
g_media_smote <- sqrt(C_Matrix_Geral_smote$byClass["Sensitivity"]*C_Matrix_Geral_smote$byClass["Specificity"])
#MCC
teste_smote <- data.frame(p_samp_smote, teste$Resposta)
class_predict_smote <- ifelse(teste_smote$p_samp_smote > 0.55,1,-1) %>% as.factor()
MCC_smote <- mcc(preds = class_predict_smote, actuals =  teste_smote$teste.Resposta);MCC1

########## BUSCA MELHOR PTCO CORTE
calcular_MCC_para_pontos_corte <- function(data, A, ponto_corte_seq) {
  melhores_MCC <- -Inf
  melhor_ponto_corte <- NA

  for (ponto_corte in ponto_corte_seq) {
    class_predict <- ifelse(data$p_samp_smote > ponto_corte, 1, 0) %>% as.factor()
    MCC <- mcc(preds = class_predict, actuals = A$teste.Resposta)

    if (MCC > melhores_MCC) {
      melhores_MCC <- MCC
      melhor_ponto_corte <- ponto_corte
    }
  }

  return(list(melhor_ponto_corte = melhor_ponto_corte, melhor_MCC = melhores_MCC))
}

teste_smote <- data.frame(p_samp_smote, teste$Resposta)
ponto_corte_seq <- seq(0, 0.99, 0.01)
resultado <- calcular_MCC_para_pontos_corte(teste_smote, teste_smote, ponto_corte_seq)

print(paste("Melhor Ponto de Corte:", resultado$melhor_ponto_corte))
print(paste("Melhor Valor de MCC:", resultado$melhor_MCC))

############ REGRESSAO LOGISTICA HIBRIDA ##################
table(treino_sub$Resposta)
tic()
data_balanced_hibrido <- SmoteClassif(Resposta ~Atraso_medio+faturaM_pagamento+Prop_pagamento+Quantidade_atrasos, treino_sub, C.perc = list('0' = 1, '1' = 2.99))
toc()
table(data_balanced_hibrido$Resposta)

tic()
modelo_h <- glm(Resposta ~Atraso_medio+faturaM_pagamento+Prop_pagamento+Quantidade_atrasos,family=binomial(link='logit'),data=data_balanced_hibrido)
toc()
summary(modelo_h)

p_samp_h <- predict(modelo_h, newdata=subset(teste), type="response")
p_samp_h
#KS
pred_modelo_geral_h = prediction(p_samp_h, teste$Resposta)

perf_modelo_geral_h = performance(pred_modelo_geral_h,"tpr","fpr")

TPRfromROCR_h=unlist(perf_modelo_geral_h@y.values)
FPRfromROCR_h=unlist(perf_modelo_geral_h@x.values)

diff_TPRFPR_h=TPRfromROCR_h-FPRfromROCR_h
KS_h=max(diff_TPRFPR_h)
cutoffAtKS_h=unlist(perf_modelo_geral_h@alpha.values)[which.max(diff_TPRFPR_h)]
print(c(KS_h, cutoffAtKS_h)) #0.5304376

#matriz de cunfusao
previsoes_binarias_h <- ifelse(p_samp_h > 0.57, '1', '-1')
C_Matrix_Geral_h <- confusionMatrix(data = as.factor(previsoes_binarias_h),
                                    reference = teste$Resposta,
                                    positive = '1',
                                    mode = "everything");C_Matrix_Geral_h
g_media_h <- sqrt(C_Matrix_Geral_h$byClass["Sensitivity"]*C_Matrix_Geral_h$byClass["Specificity"]);g_media_h
teste_h <- data.frame(p_samp_h, teste$Resposta)
class_predict_h <- ifelse(teste_h$p_samp_h > 0.57,1,-1) %>% as.factor()
MCC_h <- mcc(preds = class_predict_h, actuals =  teste_h$teste.Resposta);MCC_h

calcular_MCC_para_pontos_corte <- function(data, A, ponto_corte_seq) {
  melhores_MCC <- -Inf
  melhor_ponto_corte <- NA

  for (ponto_corte in ponto_corte_seq) {
    class_predict <- ifelse(data$p_samp_h > ponto_corte, 1, 0) %>% as.factor()
    MCC <- mcc(preds = class_predict, actuals = A$teste.Resposta)

    if (MCC > melhores_MCC) {
      melhores_MCC <- MCC
      melhor_ponto_corte <- ponto_corte
    }
  }

  return(list(melhor_ponto_corte = melhor_ponto_corte, melhor_MCC = melhores_MCC))
}

teste_2 <- data.frame(p_samp_h, teste$Resposta)
ponto_corte_seq <- seq(0, 0.99, 0.01)
resultado <- calcular_MCC_para_pontos_corte(teste_2, teste_2, ponto_corte_seq)

print(paste("Melhor Ponto de Corte:", resultado$melhor_ponto_corte))
print(paste("Melhor Valor de MCC:", resultado$melhor_MCC))

############ REGRESSAO LOGISTICA SUBAMOSTRADA ##################
table(treino$Resposta)
table(treino_sub$Resposta)
treino_sub <- treino_subamostrado_base1
treino_sub$Resposta <- as.factor(treino_sub$Resposta)
names(treino_sub)[names(treino_sub) == "Unnamed..0"] <- "X"
treino_sub <- subset(treino_sub, select = -X)
#regressao logistica no conjunto de treinamento
tic()
model <- glm(Resposta ~Atraso_medio+faturaM_pagamento+Prop_pagamento+Quantidade_atrasos,family=binomial(link='logit'),data=treino_sub)
toc()
summary(model)
#predicao no conjunto de teste
p_samp <- predict(model, newdata=subset(teste), type="response")
p_samp

#KS
pred_modelo_geral = prediction(p_samp, teste$Resposta)

perf_modelo_geral = performance(pred_modelo_geral,"tpr","fpr")

TPRfromROCR=unlist(perf_modelo_geral@y.values)
FPRfromROCR=unlist(perf_modelo_geral@x.values)

diff_TPRFPR=TPRfromROCR-FPRfromROCR
KS=max(diff_TPRFPR)
cutoffAtKS=unlist(perf_modelo_geral@alpha.values)[which.max(diff_TPRFPR)]
print(c(KS, cutoffAtKS)) #0.2702688

########## BUSCA MELHOR PTCO CORTE
calcular_MCC_para_pontos_corte <- function(data, A, ponto_corte_seq) {
  melhores_MCC <- -Inf
  melhor_ponto_corte <- NA

  for (ponto_corte in ponto_corte_seq) {
    class_predict <- ifelse(data$p_samp_h > ponto_corte, 1, 0) %>% as.factor()
    MCC <- mcc(preds = class_predict, actuals = A$teste.Resposta)

    if (MCC > melhores_MCC) {
      melhores_MCC <- MCC
      melhor_ponto_corte <- ponto_corte
    }
  }

  return(list(melhor_ponto_corte = melhor_ponto_corte, melhor_MCC = melhores_MCC))
}

teste_2 <- data.frame(p_samp, teste$Resposta)
ponto_corte_seq <- seq(0, 0.99, 0.01)
resultado <- calcular_MCC_para_pontos_corte(teste_2, teste_2, ponto_corte_seq)

print(paste("Melhor Ponto de Corte:", resultado$melhor_ponto_corte))
print(paste("Melhor Valor de MCC:", resultado$melhor_MCC))


#matriz de cunfusao
previsoes_binarias <- ifelse(p_samp > cutoffAtKS, '1', '-1')
C_Matrix_Geral <- confusionMatrix(data = as.factor(previsoes_binarias),
                                  reference = as.factor(teste$Resposta),
                                  positive = '1',
                                  mode = "everything");C_Matrix_Geral
g_media <- sqrt(C_Matrix_Geral$byClass["Sensitivity"]*C_Matrix_Geral$byClass["Specificity"]);g_media
#MCC
teste_2 <- data.frame(p_samp, teste$Resposta)
class_predict <- ifelse(teste_2$p_samp > cutoffAtKS,1,-1) %>% as.factor()
MCC1 <- mcc(preds = class_predict, actuals =  teste_2$teste.Resposta);MCC1


############ SVM DESBALANCEADA ##################
tic()
svm_radial <- fit_svm(treino,teste,prob = F, kernel ="radial")
toc()
df.svm.radial <- data.frame(Predicao = svm_radial$saida_modelo,Y = teste$Resposta)
performance.svm.radial <- fit_performance(data = df.svm.radial,prob = F)
############ SVM SMOTE ##################
tic()
data_balanced_smote <- SmoteClassif(Resposta ~ ., treino, C.perc = list('-1' = 1, '1' = 3.52))
svm_radial_smote <- fit_svm(data_balanced_smote,teste,prob = F, kernel ="radial")
toc()
df.svm.radial_smote <- data.frame(Predicao = svm_radial_smote$saida_modelo,Y = teste$Resposta)
performance.svm.radial_smote <- fit_performance(data = df.svm.radial_smote,prob = F)
############ SVM SUB ##################
tic()
svm_radial_sub <- fit_svm(data_balanced_sub,teste,prob = F, kernel ="radial")
toc()
df.svm.radial_sub <- data.frame(Predicao = svm_radial_sub$saida_modelo,Y = teste$Resposta)
performance.svm.radial_sub <- fit_performance(data = df.svm.radial_sub,prob = F)
############ SVM HYBRID ##################
tic()
svm_radial_hibrido <- fit_svm(data_balanced_hibrido,teste,prob = F, kernel ="radial")
toc()
df.svm.radial_hibrido <- data.frame(Predicao = svm_radial_hibrido$saida_modelo,Y = teste$Resposta)
performance.svm.radial_hibrido <- fit_performance(data = df.svm.radial_hibrido,prob = F)
############ SVM SENSIVEL AO CUSTO ##################
tic()
svm_peso <- SVM_weight(treino,teste,kernel = "radial", prob = TRUE, peso_min = 3,
                       peso_max = 4, range = 0.2)
toc()
svm_peso_resultados <- SVM_weight_perf(svm_peso,prob = FALSE)
svm_peso_resultados$G_Mean
svm_peso_resultados$F1
svm_peso_resultados$MCC



#N?o lembro qual era o certo, se eram esses dois debaixo ou o SVM SUB e HYBRID de cima -> um dos dois da erro no conjunto subamostrado, mas ? um dos dois
############ SVM subamostragem ##################
#treino_sub <- subset(treino_sub, select = -X)
tic()
svm_radial_sub <- fit_svm(treino_sub,teste,prob = F, kernel ="radial")
toc()
df.svm.radial_sub <- data.frame(Predicao = svm_radial_sub$saida_modelo,Y = teste$Resposta)
performance.svm.radial_sub <- fit_performance(data = df.svm.radial_sub,prob = F);performance.svm.radial_sub
############ SVM HIBRIDO ##################
#data_balanced_hibrido <- subset(data_balanced_hibrido, select = -X)
tic()
svm_radial_h <- fit_svm(data_balanced_hibrido,teste,prob = F, kernel ="radial")
toc()
df.svm.radial_h <- data.frame(Predicao = svm_radial_h$saida_modelo,Y = teste$Resposta)
performance.svm.radial_h <- fit_performance(data = df.svm.radial_h,prob = F);performance.svm.radial_h
